# Lab 2: Numerical solutions of ODEs

In this Lab we will work with the implementations of numerical schemes provided by Jacques, and use these to solve problems from Chapter 8 of Boyce & DiPrima.

## 1. The numerical schemes

Here is the code provided by Jacques in the notebook `ODEmanySchemes`:

In [1]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook


# Euler scheme
def Euler(vectorField,times,initialConditions):
    n = vectorField(initialConditions,times[0]).size
    x = np.zeros((times.size,n))
    x[0,:] = initialConditions
    for k, t in enumerate(times[:-1]):
        x[k+1,:] = x[k,:]+(times[k+1]-t)*vectorField(x[k,:],t)
    return x

# Heun scheme
def Heun(vectorField,times,initialConditions):
    n = vectorField(initialConditions,times[0]).size
    x = np.zeros((times.size,n))
    x[0,:] = initialConditions
    for k, t in enumerate(times[:-1]):
        dt = times[k+1]-t
        f1 = vectorField(x[k,:],t)
        f2 = vectorField(x[k,:]+dt*f1,t+dt)
        x[k+1,:] = x[k,:]+0.5*dt*(f1+f2)
    return x

# Runge--Kutta 4 scheme
def RungeKutta(vectorField,times,initialConditions):
    n = vectorField(initialConditions,times[0]).size
    x = np.zeros((times.size,n))
    x[0,:] = initialConditions
    for k, t in enumerate(times[:-1]):
        dt=times[k+1]-t
        f1 = vectorField(x[k,:],t)
        f2 = vectorField(x[k,:]+0.5*dt*f1,t+0.5*dt)
        f3 = vectorField(x[k,:]+0.5*dt*f2,t+0.5*dt)
        f4 =vectorField(x[k,:]+dt*f3,t+dt)
        x[k+1,:] = x[k,:]+dt/6*(f1+2*f2+2*f3+f4)
    return x

# Adams--Bashforth 2 (here needing a fixed timestep)
def AdamsBashforth2(vectorField,initialTime,finalTime,nSteps,initialConditions):
    n = 1#vectorField(initialConditions,initialTime).size
    dt=(finalTime-initialTime)/nSteps
    t=np.linspace(initialTime,finalTime,nSteps+1)
    x = np.zeros((nSteps+1,n))
    x[0,:] = initialConditions
    # First step using Euler
    vField = vectorField(x[0,:],initialTime)
    x[1,:] = x[0,:]+dt*vField
    # Other steps
    for k in range(1,nSteps):
        vFieldOld = vField
        vField = vectorField(x[k,:],initialTime+k*dt)
        x[k+1,:] = x[k,:]+(1.5*vField-0.5*vFieldOld)*dt
       
    return x, t 

<div class="alert alert-info">
    <h3>Exercise 1.1</h3>

**(a)** What is the purpose of `enumerate(times[:-1])` as used in the first three functions?

**(b)** In AdamsBashforth2, why `+1` in `t=np.linspace(initialTime,finalTime,nSteps+1)`?

</div>

**Answers**

**(a)** you have a list of all the numbers excet the last times step. Since we allways get dt=times[k+1]-t we dont want a index error.

**(b)**Because it is a 2nd and we need a 2nd linestep.

Boyce & DiPrima Chapter 8 uses the equation $$y'=1-t+4y, \\y(0)=1$$ throughout to illustrate these different numerical methods. 

Here we define a function `eq7_dy_dt` that gives the value of the derivative $y'$ at a given point, then use the `Euler` method to solve the equation numerically with step size $h=0.01$:

<span class="label label-danger">Task</span>
Why is it `np.linspace(0,2,201)` and not `np.linspace(0,2,200)`? Try changing the code and see what happens.

In [4]:
def eq7_dy_dt(y, t):
    return 1 - t + 4*y

times = np.linspace(0,2,201)
len(times),len(times[:-1])
eq7_euler = Euler(eq7_dy_dt, times, 1)
eq7_euler

array([[1.00000000e+00],
       [1.05000000e+00],
       [1.10190000e+00],
       [1.15577600e+00],
       [1.21170704e+00],
       [1.26977532e+00],
       [1.33006633e+00],
       [1.39266899e+00],
       [1.45767575e+00],
       [1.52518278e+00],
       [1.59529009e+00],
       [1.66810169e+00],
       [1.74372576e+00],
       [1.82227479e+00],
       [1.90386578e+00],
       [1.98862041e+00],
       [2.07666523e+00],
       [2.16813184e+00],
       [2.26315711e+00],
       [2.36188340e+00],
       [2.46445873e+00],
       [2.57103708e+00],
       [2.68177856e+00],
       [2.79684971e+00],
       [2.91642370e+00],
       [3.04068064e+00],
       [3.16980787e+00],
       [3.30400018e+00],
       [3.44346019e+00],
       [3.58839860e+00],
       [3.73903454e+00],
       [3.89559592e+00],
       [4.05831976e+00],
       [4.22745255e+00],
       [4.40325065e+00],
       [4.58598068e+00],
       [4.77591991e+00],
       [4.97335670e+00],
       [5.17859097e+00],
       [5.39193461e+00],


We can check that the final value this gives (corresponding to $t=2$) agrees with the value $3029.3279$ given in Table 8.1.1 in the book:

In [5]:
eq7_euler[-1]

array([3029.32787693])

In fact, by creating a `DataFrame` object using the [Pandas package](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html), we can see the whole table of values:

In [ ]:
from pandas import DataFrame
a = DataFrame(data = eq7_euler, index = times, columns = ["h=0.01"])
a

<div class="alert alert-info">
    <h3>Exercise 1.2</h3>

**(a)** Write code to check the final value in the $h=0.001$ column, $3484.1608$.

**(b)** Check that applying the Heun method with $h=0.01$ gives $3532.8789$ when $t=2$, as in Table 8.2.1 of the book.

</div>

In [ ]:
def eq8_dy_dt(y, t):
    return 1 - t + 4*y

times = np.linspace(0,2,2001)
eq8_euler = Euler(eq8_dy_dt, times, 1)
eq8_euler[-1]


In [ ]:
def eq9_dy_dt(y, t):
    return 1 - t + 4*y

times = np.linspace(0,2,201)
eq9_heun = Heun(eq9_dy_dt, times, 1)
eq9_heun[-1]


## 2. Standardising the arguments

Notice that the first three methods (`Euler`, `Heun` and `RungeKutta`) have the same pattern of arguments, but the fourth method (`AdamsBashforth2`) is different. Furthermore, none of the four methods accepts a step size (e.g. $h=0.05$), which is the parameter that we would like to vary.

In this section, we will fix these issues by creating four new versions of the functions which can be used interchangeably.

The first step is to create a new function that will create the `times` needed in the first three methods.

Here is a basic implementation of a `timesteps` function, which generates an array of numbers starting at the value `start` and increasing with step size `h` until the value `stop` is reached.

In [ ]:
import math
def timesteps(start, stop, h):
    num_steps = math.ceil((stop - start)/h)
    return np.linspace(start, start+num_steps*h, num_steps+1)

(timesteps(0, 0.5, h = 0.1))

<span class="label label-danger">Task</span>
Check that this function does indeed work as expected, by trying it out with different inputs. Think about different tricky cases which might break the code, and try those!

<div class="alert alert-info">
    <h3>Exercise 2.1</h3>

Produce new functions `Euler_step`, `Heun_step`, `RungeKutta_step` and `AdamsBashforth2_step`, which use the four functions provided by Jacques to solve a given equation numerically. Each of the new functions should take in the arguments:

* `vectorField` - a function defining the differential equation
* `start` - the initial time
* `stop` - the end time
* `h` - the step size
* `ics` - the intial conditions

and return both the values of the numerical solution and the list of time steps (like in the `AdamsBashforth2` function).

</div>

Note - you should **not** copy and modify the original functions given above. Instead, you should call those functions from within your own ones, having worked out the appropriate values to pass to them.

In [ ]:
def Euler_step(vectorField, start, stop, h, ics):
    t = timesteps(start, stop, h)
    x = Euler(vectorField, t, ics)
    return x, t

def Heun_step(vectorField, start, stop, h, ics):
    t = timesteps(start, stop, h)
    x = Heun(vectorField, t, ics)
    return x, t

def RungeKutta_step(vectorField, start, stop, h, ics):
    t = timesteps(start, stop, h)
    x = RungeKutta(vectorField, t, ics)
    return x, t

def AdamsBashforth2_step(vectorField, start, stop, h, ics):
    t = timesteps(start, stop, h)
    num_steps = len(t)-1
    return AdamsBashforth2(vectorField,start,stop,num_steps,ics)
    

## 3. Building tables

Now that we have these four functions with consistent arguments, we can easily replicate all the tables in Boyce & DiPrima, and solve further exercises.

For instance, we can produce one column of Table 8.1.1 with the following code:

In [ ]:
eq7_E_values, eq7_E_times = Euler_step(eq7_dy_dt, 0, 2, h = 0.01, ics = 1)
DataFrame(data = eq7_E_values, index = eq7_E_times, columns = ["h=0.01"])

### Detour: list comprehensions and joining DataFrames

It would be possible at this point to copy/paste the above code, and make small changes to get other columns of the table. However, we can be a bit more efficient than that by making use of **list comprehensions**.

These are very similar to the standard mathematical notation for sets, e.g. $$\left\{x^2\,:\,x\in\{1,2,3\}\right\} = \{1^2, 2^2, 3^2\}$$ could be written in Python as:

In [ ]:
[x ** 2 for x in [1,2,3]]

Here we use a list comprehension to produce a DataFrame where the column of values is given by a certain power of the row index:

In [ ]:
def x_to_power(n):
    return DataFrame(data = [x ** n for x in range(11)], columns = ["x^"+str(n)])

x_to_power(2)

<span class="label label-danger">Task</span>
Try changing `range(5)` in the definition of `x_to_power` so that the squares of 1 to 10 are printed instead.

We can use list comprehensions to produce many such DataFrames, each one with a different power:

In [ ]:
dfs = [x_to_power(n) for n in [2,3,4]]
dfs[2] # try changing 0 to see the other DataFrames

To put these columns into a single DataFrame, we can use the `join` function:

In [ ]:
dfs[0].join(dfs[1])

In fact, it can be used to join multiple DataFrames at the same time:

In [ ]:
dfs[0].join(dfs[1:])

Putting that all together, here we build up a table in one chunk:

In [ ]:
dfs2 = [x_to_power(n) for n in [0.5, 2, 3, 4]]
dfs2[0].join(dfs2[1:])

Note that we can also use the `filter` function to pick out certain rows from the DataFrame - we use the `items` argument to specify the values of the row index, and `axis=0` to specify that it's rows rather than columns that we are selecting:

In [ ]:
big_df = dfs2[0].join(dfs2[1:])
big_df.filter(items=[2, 4], axis=0)

<div class="alert alert-info">
    <h3>Exercise 3.1</h3>

**(a)** Write a `produce_df` function that takes in a `method` (which will be a function, like `Euler_step`) and all the arguments from Exercise 2.1, and returns a DataFrame showing the result of applying that method with the given data. In particular, `produce_df(Euler_step, eq7_dy_dt, 0, 2, 0.01, 1)` should produce the same table that we saw just before the Detour.

**(b)** Use the `produce_df` function, and the joining technique outlined above, to replicate Table 8.1.1 from Boyce & DiPrima.

</div>

In [ ]:
def produce_df(method,vectorField, start, stop, h, ics):
    x, t = method(vectorField, start, stop, h, ics)
    return DataFrame(data = x, index = np.round(t, 3), columns = [str(method)+" h="+str(h)])

produce_df(Euler_step, eq7_dy_dt, 0, 2, 0.01, 1)

In [ ]:
book_table = [produce_df(Euler_step, eq7_dy_dt, 0, 2, h, 1) for h in [0.05,0.025,0.01,0.001]]
final_table = book_table[0].join(book_table[1:])
final_table

We can also easily plot the different columns of a DataFrame using the `matplotlib.pyplot` library that we used at the end of Lab 1:

In [ ]:
import matplotlib.pyplot as plt
big_df.plot()

<div class="alert alert-info">
    <h3>Exercise 3.2</h3>

Using the DataFrame constructed in the previous exercise, produce a plot of the different solutions from Table 8.1.1

</div>

In [ ]:
import matplotlib.pyplot as plt
final_table.plot()

The plot looks strangely patchy. This is because the DataFrame actually has some missing values, as we can see in this example:

In [ ]:
example_dfs = [produce_df(Euler_step, eq7_dy_dt, 0, 2, h, 1) for h in [0.1, 0.01]]
example_table = example_dfs[0].join(example_dfs[1:])
example_table.plot()

The reason for this is that in the original data, some of the times actually have small rounding errors:

In [ ]:
values, times = Euler_step(eq7_dy_dt, 0, 2, h = 0.01, ics = 1)

# Print the first 50 times - they look OK
print(times[:50])

# Forcing numpy to print more decimal places, we see the problem:
with np.printoptions(precision=20):
    print(times[:50])

Since the times are used to join together corresponding rows of the DataFrames, this causes a problem. We can fix it by setting the `index` of the DataFrame to be a rounded version of the time, using `np.round`:

In [ ]:
def produce_df(method, vectorField, start, stop, h, ics):
    values, times = method(vectorField, start, stop, h, ics)
    return DataFrame(data = values, index = np.round(times, 3), columns = ["h = "+str(h)])

<span class="label label-danger">Task</span>
Fix your `produce_df` function and check that the plots now work.

<div class="alert alert-info">
    <h3>Exercise 3.3</h3>

Put all of this together to produce a table like the following one, and to plot the different columns.

Note: the final column shows the exact solution, $y=\frac14 t-\frac{3}{16}+\frac{19}{16}e^{4t}$.

</div>

|     | Euler, h=0.01 | Heun, h=0.01 | RK, h=0.01  | AB, h=0.01  | Exact       |
|-----|---------------|--------------|-------------|-------------|-------------|
| 0.0 | 1.000000      | 1.000000     | 1.000000    | 1.000000    | 1.000000    |
| 0.1 | 1.595290      | 1.608858     | 1.609042    | 1.607223    | 1.609042    |
| 0.2 | 2.464459      | 2.504783     | 2.505330    | 2.501929    | 2.505330    |
| 0.3 | 3.739035      | 3.828915     | 3.830139    | 3.824042    | 3.830139    |
| 0.4 | 5.613712      | 5.791791     | 5.794226    | 5.783604    | 5.794226    |
| 0.5 | 8.376686      | 8.707464     | 8.712004    | 8.693881    | 8.712004    |
| 1.0 | 60.037126     | 64.830722    | 64.897798   | 64.679828   | 64.897803   |
| 1.5 | 426.408176    | 478.515883   | 479.259133  | 477.028224  | 479.259192  |
| 2.0 | 3029.327877   | 3532.878861  | 3540.199525 | 3519.137591 | 3540.200110 |


In [ ]:
book_table = [produce_df(method, eq7_dy_dt, 0, 2, 0.1, 1) for method in [Euler_step,Heun_step,RungeKutta_step,AdamsBashforth2_step]]
a = book_table[0].rename(columns = {"<function Euler_step at 0x7ff048cbd2f0> h=0.1" :"Euler, h=0.1"})
b = book_table[1].rename(columns = {"<function Heun_step at 0x7ff048c8db70> h=0.1" :"Heun, h=0.1"})
c = book_table[2].rename(columns = {"<function RungeKutta_step at 0x7ff04d059158> h=0.1" :"RK, h=0.1"})
d = book_table[3].rename(columns = {"<function AdamsBashforth2_step at 0x7ff04b221ea0> h=0.1" :"AB, h=0.1"})
def exact():
    return  DataFrame(data = [((1/4*x)-(3/16)+(19/16*np.exp(4*x))) for x in timesteps(0,2,0.1)], index = np.round(timesteps(0,2,0.1), 3), columns = ["Exact"])
e = exact()

    
final = a.join(b.join(c.join(d.join(e))))
final.iloc[[0,1,2,3,4,5,6,7,8,9,10,15,20]]

## 4. Solving further equations

<div class="alert alert-info">
    <h3>Exercise 4.1</h3>

Use the same approach to produce a table and a plot of the different numerical solutions to the following initial value problems:

**(a)** $y'=2t+e^{-ty}$, $y(0)=1$,

**(b)** $y'=(t^2-y^2)\sin y$, $y(0)=-1$.

Note: these appear as questions in Chapter 8 of Boyce & DiPrima, so if you use $h=0.05$ you can check your results against the solutions in the book. They are questions 4 and 6 in sections 8.1, 8.2 and 8.3 of Boyce & DiPrima.

</div>

Hint: remember you can use SymPy to find exact solutions to compare against, as in Lab 1. However, you will notice here that the exact solutions are not very helpful!


<div class="alert alert-info">
    <h3>Exercise 4.2</h3>

Solve Problem 6 in section 8.5 of Boyce & DiPrima (p481):

$$ x'=\exp(-x+y)-\cos x, \quad y'=\sin(x-3y); \quad x(0)=1, y(0)=2 $$

finding the approximate solutions at $t=0.2, 0.4, 0.6, 0.8, 1.0$ using:

**(a)** the Euler method with $h=0.1$

**(b)** Runge-Kutta with $h=0.2$

**(c)** Runge-Kutta with $h=0.1$

</div>